# ETL Premier League 2018/2019

This work's contents encompass on-running Premier League 2018/2019 season with an aim of spotting weekly position among four contestants: Liverpool F.C., Manchester United, Manchester City and Tottenham. This code below shows the breakdown matchday's result and its aggregations makes way for securing their position in league bracket. 

In [6]:
# importing modules

import json
import requests
from datetime import datetime, date
import pandas as pd          # to read data in python with dataframe format
import sqlalchemy            # Toolkit and Object Relational Mapper that gives full power and flexibility of SQL
import matplotlib.pyplot as plt     # Visualization
import psycopg2
import psycopg2.extras
import http.client
from pandas.io.json import json_normalize
from sqlalchemy import create_engine

In [7]:
# Accessing url for loading data in json format

url = 'http://api.football-data.org/v2/competitions/PL/matches'
headers = { 'X-Auth-Token': '3ae6b58cc03247309be129f237b28a4d' }
response = requests.get(url, headers=headers)
data = response.text
parsed = json.loads(data)

match_number = parsed['matches']

In [8]:
# Finding matchday in match no. 1 (EPL first official match)
match_number[0]['matchday']

1

In [20]:
# Finding last match in finished terms
i = 0
for i in range(380):
    if match_number[i]['status'] == 'SCHEDULED':
        break
print('# EPL match which status remains scheduled is : #',i)

# EPL match which status remains scheduled is : # 360


In [21]:
# Knowing match no. 1 score 
match_number[0]['score']

{'winner': 'HOME_TEAM',
 'duration': 'REGULAR',
 'fullTime': {'homeTeam': 2, 'awayTeam': 1},
 'halfTime': {'homeTeam': 1, 'awayTeam': 0},
 'extraTime': {'homeTeam': None, 'awayTeam': None},
 'penalties': {'homeTeam': None, 'awayTeam': None}}

In [33]:
# Finding first EPL match which both sides ended draw
i = 0
for i in range(380):
    if match_number[i]['score']['winner'] == 'DRAW':
        break
print('# First EPL match which both teams earn a single point on 1 is : #',i)
print('Home Team :', match_number[i]['homeTeam']['name'])
print('Away Team :', match_number[i]['awayTeam']['name'])

# First EPL match which both teams earn a single point on 1 is : # 6
Home Team : Wolverhampton Wanderers FC
Away Team : Everton FC


In [36]:
# Knowing which team winning the match no. 1

if match_number[0]['score']['winner'] == 'HOME_TEAM':
    print (match_number[0]['homeTeam']['name'])

Manchester United FC


In [37]:
# Picking match date and slicing it
re = match_number[0]['utcDate']
print(re[slice(10)])


2018-08-10


Now, let's get straight into dict

# Creating EPL Matchday Tabulation

As the season draws near to the end with only Liverpool and Manchester City keeping their momentum alive in title chase, now the fixture's results are possible to get into breakdown in details. 

In [38]:
# Creating dictionary from json files
foot = {}
r = 0

# Add values to `new_dict` using for loop
for n in parsed['matches']:
    foot[r] = {}
    foot[r]['matchday'] = n['matchday']
    foot[r]['status'] = n['status']
    foot[r]['hometeam'] = n['homeTeam']['name']
    foot[r]['awayteam'] = n['awayTeam']['name']
    foot[r]['date'] = n['utcDate']
    foot[r]['winner'] = n['score']['winner']
    r +=1
    
    if (n['status'] != 'FINISHED'):
        break
    
print(foot)

{0: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'Manchester United FC', 'awayteam': 'Leicester City FC', 'date': '2018-08-10T19:00:00Z', 'winner': 'HOME_TEAM'}, 1: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'Newcastle United FC', 'awayteam': 'Tottenham Hotspur FC', 'date': '2018-08-11T11:30:00Z', 'winner': 'AWAY_TEAM'}, 2: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'Fulham FC', 'awayteam': 'Crystal Palace FC', 'date': '2018-08-11T14:00:00Z', 'winner': 'AWAY_TEAM'}, 3: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'Huddersfield Town AFC', 'awayteam': 'Chelsea FC', 'date': '2018-08-11T14:00:00Z', 'winner': 'AWAY_TEAM'}, 4: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'Watford FC', 'awayteam': 'Brighton & Hove Albion FC', 'date': '2018-08-11T14:00:00Z', 'winner': 'HOME_TEAM'}, 5: {'matchday': 1, 'status': 'FINISHED', 'hometeam': 'AFC Bournemouth', 'awayteam': 'Cardiff City FC', 'date': '2018-08-11T14:00:00Z', 'winner': 'HOME_TEAM'}, 6: {'matchday': 1, 'status'

In [39]:
fixture = pd.DataFrame.from_dict(foot,orient='columns')
fixture

,0,1,2,3,4,5,6,7,8,9,...,351,352,353,354,355,356,357,358,359,360
awayteam,Leicester City FC,Tottenham Hotspur FC,Crystal Palace FC,Chelsea FC,Brighton & Hove Albion FC,Cardiff City FC,Everton FC,West Ham United FC,Burnley FC,Manchester City FC,...,West Ham United FC,Cardiff City FC,AFC Bournemouth,Everton FC,Wolverhampton Wanderers FC,Newcastle United FC,Arsenal FC,Manchester City FC,Chelsea FC,Burnley FC
date,2018-08-10T19:00:00Z,2018-08-11T11:30:00Z,2018-08-11T14:00:00Z,2018-08-11T14:00:00Z,2018-08-11T14:00:00Z,2018-08-11T14:00:00Z,2018-08-11T16:30:00Z,2018-08-12T12:30:00Z,2018-08-12T12:30:00Z,2018-08-12T15:00:00Z,...,2019-04-27T11:30:00Z,2019-04-27T14:00:00Z,2019-04-27T14:00:00Z,2019-04-27T14:00:00Z,2019-04-27T14:00:00Z,2019-04-27T16:30:00Z,2019-04-28T11:00:00Z,2019-04-28T13:05:00Z,2019-04-28T15:30:00Z,2019-05-03T19:00:00Z
hometeam,Manchester United FC,Newcastle United FC,Fulham FC,Huddersfield Town AFC,Watford FC,AFC Bournemouth,Wolverhampton Wanderers FC,Liverpool FC,Southampton FC,Arsenal FC,...,Tottenham Hotspur FC,Fulham FC,Southampton FC,Crystal Palace FC,Watford FC,Brighton & Hove Albion FC,Leicester City FC,Burnley FC,Manchester United FC,Everton FC
matchday,1,1,1,1,1,1,1,1,1,1,...,36,36,36,36,36,36,36,36,36,37
status,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,...,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,FINISHED,SCHEDULED
winner,HOME_TEAM,AWAY_TEAM,AWAY_TEAM,AWAY_TEAM,HOME_TEAM,HOME_TEAM,DRAW,HOME_TEAM,DRAW,AWAY_TEAM,...,AWAY_TEAM,HOME_TEAM,DRAW,DRAW,AWAY_TEAM,DRAW,HOME_TEAM,AWAY_TEAM,DRAW,None


In [40]:
# Transposing EPL fixtures
fixture = fixture.transpose()
fixture.head()

,awayteam,date,hometeam,matchday,status,winner
0,Leicester City FC,2018-08-10T19:00:00Z,Manchester United FC,1,FINISHED,HOME_TEAM
1,Tottenham Hotspur FC,2018-08-11T11:30:00Z,Newcastle United FC,1,FINISHED,AWAY_TEAM
2,Crystal Palace FC,2018-08-11T14:00:00Z,Fulham FC,1,FINISHED,AWAY_TEAM
3,Chelsea FC,2018-08-11T14:00:00Z,Huddersfield Town AFC,1,FINISHED,AWAY_TEAM
4,Brighton & Hove Albion FC,2018-08-11T14:00:00Z,Watford FC,1,FINISHED,HOME_TEAM


In [69]:
cols = fixture.columns.tolist()
cols

['awayteam', 'date', 'hometeam', 'matchday', 'status', 'winner']

In [41]:
# Re-arrange columns 
fixture = fixture[['date','matchday','hometeam','awayteam','status','winner']]

# Formatting EPL date & time with pd.to_datetime
fixture['date'] = pd.to_datetime(fixture['date'],format='%Y-%m-%d')

# Bringing EPL completed matches in first five rows into screen
fixture.head()

,date,matchday,hometeam,awayteam,status,winner
0,2018-08-10 19:00:00,1,Manchester United FC,Leicester City FC,FINISHED,HOME_TEAM
1,2018-08-11 11:30:00,1,Newcastle United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
2,2018-08-11 14:00:00,1,Fulham FC,Crystal Palace FC,FINISHED,AWAY_TEAM
3,2018-08-11 14:00:00,1,Huddersfield Town AFC,Chelsea FC,FINISHED,AWAY_TEAM
4,2018-08-11 14:00:00,1,Watford FC,Brighton & Hove Albion FC,FINISHED,HOME_TEAM


In [42]:
# Showing most-current EPL matches in last five rows
fixture.tail()

,date,matchday,hometeam,awayteam,status,winner
356,2019-04-27 16:30:00,36,Brighton & Hove Albion FC,Newcastle United FC,FINISHED,DRAW
357,2019-04-28 11:00:00,36,Leicester City FC,Arsenal FC,FINISHED,HOME_TEAM
358,2019-04-28 13:05:00,36,Burnley FC,Manchester City FC,FINISHED,AWAY_TEAM
359,2019-04-28 15:30:00,36,Manchester United FC,Chelsea FC,FINISHED,DRAW
360,2019-05-03 19:00:00,37,Everton FC,Burnley FC,SCHEDULED,None


In [43]:
# Selecting four teams for getting deeper of their season current records
array = ['Manchester United FC', 'Liverpool FC','Tottenham Hotspur FC','Manchester City FC']

# Finding team name equal to values of hometeam or awayteam's fixtures
ManUtd = fixture.loc[(fixture['hometeam'] == 'Manchester United FC') | (fixture['awayteam'] == 'Manchester United FC')]
ManCity = fixture.loc[(fixture['hometeam'] == 'Manchester City FC') | (fixture['awayteam'] == 'Manchester City FC')]
TotHam = fixture.loc[(fixture['hometeam'] == 'Tottenham Hotspur FC') | (fixture['awayteam'] == 'Tottenham Hotspur FC')]
Liv = fixture.loc[(fixture['hometeam'] == 'Liverpool FC') | (fixture['awayteam'] == 'Liverpool FC')]


In [44]:
# Manchester United FC season records 
ManUtd

,date,matchday,hometeam,awayteam,status,winner
0,2018-08-10 19:00:00,1,Manchester United FC,Leicester City FC,FINISHED,HOME_TEAM
18,2018-08-19 15:00:00,2,Brighton & Hove Albion FC,Manchester United FC,FINISHED,HOME_TEAM
29,2018-08-27 19:00:00,3,Manchester United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
38,2018-09-02 15:00:00,4,Burnley FC,Manchester United FC,FINISHED,AWAY_TEAM
46,2018-09-15 16:30:00,5,Watford FC,Manchester United FC,FINISHED,AWAY_TEAM
52,2018-09-22 14:00:00,6,Manchester United FC,Wolverhampton Wanderers FC,FINISHED,DRAW
60,2018-09-29 11:30:00,7,West Ham United FC,Manchester United FC,FINISHED,HOME_TEAM
76,2018-10-06 16:30:00,8,Manchester United FC,Newcastle United FC,FINISHED,HOME_TEAM
80,2018-10-20 11:30:00,9,Chelsea FC,Manchester United FC,FINISHED,DRAW
98,2018-10-28 16:00:00,10,Manchester United FC,Everton FC,FINISHED,HOME_TEAM


In [45]:
# Manchester City FC season records 
ManCity

,date,matchday,hometeam,awayteam,status,winner
9,2018-08-12 15:00:00,1,Arsenal FC,Manchester City FC,FINISHED,AWAY_TEAM
16,2018-08-19 12:30:00,2,Manchester City FC,Huddersfield Town AFC,FINISHED,HOME_TEAM
20,2018-08-25 11:30:00,3,Wolverhampton Wanderers FC,Manchester City FC,FINISHED,DRAW
36,2018-09-01 16:30:00,4,Manchester City FC,Newcastle United FC,FINISHED,HOME_TEAM
41,2018-09-15 14:00:00,5,Manchester City FC,Fulham FC,FINISHED,HOME_TEAM
55,2018-09-22 14:00:00,6,Cardiff City FC,Manchester City FC,FINISHED,AWAY_TEAM
62,2018-09-29 14:00:00,7,Manchester City FC,Brighton & Hove Albion FC,FINISHED,HOME_TEAM
79,2018-10-07 15:30:00,8,Liverpool FC,Manchester City FC,FINISHED,DRAW
82,2018-10-20 14:00:00,9,Manchester City FC,Burnley FC,FINISHED,HOME_TEAM
99,2018-10-29 20:00:00,10,Tottenham Hotspur FC,Manchester City FC,FINISHED,AWAY_TEAM


In [46]:
# Liverpool FC season records 
Liv

,date,matchday,hometeam,awayteam,status,winner
7,2018-08-12 12:30:00,1,Liverpool FC,West Ham United FC,FINISHED,HOME_TEAM
19,2018-08-20 19:00:00,2,Crystal Palace FC,Liverpool FC,FINISHED,AWAY_TEAM
25,2018-08-25 16:30:00,3,Liverpool FC,Brighton & Hove Albion FC,FINISHED,HOME_TEAM
30,2018-09-01 11:30:00,4,Leicester City FC,Liverpool FC,FINISHED,AWAY_TEAM
40,2018-09-15 11:30:00,5,Tottenham Hotspur FC,Liverpool FC,FINISHED,AWAY_TEAM
51,2018-09-22 14:00:00,6,Liverpool FC,Southampton FC,FINISHED,HOME_TEAM
67,2018-09-29 16:30:00,7,Chelsea FC,Liverpool FC,FINISHED,DRAW
79,2018-10-07 15:30:00,8,Liverpool FC,Manchester City FC,FINISHED,DRAW
87,2018-10-20 16:30:00,9,Huddersfield Town AFC,Liverpool FC,FINISHED,AWAY_TEAM
91,2018-10-27 14:00:00,10,Liverpool FC,Cardiff City FC,FINISHED,HOME_TEAM


In [47]:
# Tottenham Hotspur FC season records 
TotHam

,date,matchday,hometeam,awayteam,status,winner
1,2018-08-11 11:30:00,1,Newcastle United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
11,2018-08-18 14:00:00,2,Tottenham Hotspur FC,Fulham FC,FINISHED,HOME_TEAM
29,2018-08-27 19:00:00,3,Manchester United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
39,2018-09-02 15:00:00,4,Watford FC,Tottenham Hotspur FC,FINISHED,HOME_TEAM
40,2018-09-15 11:30:00,5,Tottenham Hotspur FC,Liverpool FC,FINISHED,AWAY_TEAM
57,2018-09-22 16:30:00,6,Brighton & Hove Albion FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
66,2018-09-29 14:00:00,7,Huddersfield Town AFC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
71,2018-10-06 14:00:00,8,Tottenham Hotspur FC,Cardiff City FC,FINISHED,HOME_TEAM
85,2018-10-20 14:00:00,9,West Ham United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM
99,2018-10-29 20:00:00,10,Tottenham Hotspur FC,Manchester City FC,FINISHED,AWAY_TEAM


In [48]:
# Setting initial instance for total points acquired for Tottenham Hostpur FC
TotHam['total_point'] = 0

# Code for generating point based on each match result which Tottenham Hostpur FC attended to
TotHam.loc[(TotHam['hometeam']=='Tottenham Hotspur FC') & (TotHam['winner']=='HOME_TEAM'),'point'] = 3
TotHam.loc[(TotHam['hometeam']=='Tottenham Hotspur FC') & (TotHam['winner']=='AWAY_TEAM'),'point'] = 0
TotHam.loc[(TotHam['awayteam']=='Tottenham Hotspur FC') & (TotHam['winner']=='AWAY_TEAM'),'point'] = 3
TotHam.loc[(TotHam['awayteam']=='Tottenham Hotspur FC') & (TotHam['winner']=='HOME_TEAM'),'point'] = 0
TotHam.loc[(TotHam['winner']=='DRAW'),'point'] = 1

# Showing point obtained per match
TotHam        

C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Rizdi\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Rizdi\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas

,date,matchday,hometeam,awayteam,status,winner,total_point,point
1,2018-08-11 11:30:00,1,Newcastle United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,0,3.0
11,2018-08-18 14:00:00,2,Tottenham Hotspur FC,Fulham FC,FINISHED,HOME_TEAM,0,3.0
29,2018-08-27 19:00:00,3,Manchester United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,0,3.0
39,2018-09-02 15:00:00,4,Watford FC,Tottenham Hotspur FC,FINISHED,HOME_TEAM,0,0.0
40,2018-09-15 11:30:00,5,Tottenham Hotspur FC,Liverpool FC,FINISHED,AWAY_TEAM,0,0.0
57,2018-09-22 16:30:00,6,Brighton & Hove Albion FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,0,3.0
66,2018-09-29 14:00:00,7,Huddersfield Town AFC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,0,3.0
71,2018-10-06 14:00:00,8,Tottenham Hotspur FC,Cardiff City FC,FINISHED,HOME_TEAM,0,3.0
85,2018-10-20 14:00:00,9,West Ham United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,0,3.0
99,2018-10-29 20:00:00,10,Tottenham Hotspur FC,Manchester City FC,FINISHED,AWAY_TEAM,0,0.0


In [49]:
# Tottenham Hostpur FC point accumulation 
TotHam['total_point'] = TotHam['point'].cumsum()

# Showing total points obtained so far .....
TotHam

C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date,matchday,hometeam,awayteam,status,winner,total_point,point
1,2018-08-11 11:30:00,1,Newcastle United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,3.0,3.0
11,2018-08-18 14:00:00,2,Tottenham Hotspur FC,Fulham FC,FINISHED,HOME_TEAM,6.0,3.0
29,2018-08-27 19:00:00,3,Manchester United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,9.0,3.0
39,2018-09-02 15:00:00,4,Watford FC,Tottenham Hotspur FC,FINISHED,HOME_TEAM,9.0,0.0
40,2018-09-15 11:30:00,5,Tottenham Hotspur FC,Liverpool FC,FINISHED,AWAY_TEAM,9.0,0.0
57,2018-09-22 16:30:00,6,Brighton & Hove Albion FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,12.0,3.0
66,2018-09-29 14:00:00,7,Huddersfield Town AFC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,15.0,3.0
71,2018-10-06 14:00:00,8,Tottenham Hotspur FC,Cardiff City FC,FINISHED,HOME_TEAM,18.0,3.0
85,2018-10-20 14:00:00,9,West Ham United FC,Tottenham Hotspur FC,FINISHED,AWAY_TEAM,21.0,3.0
99,2018-10-29 20:00:00,10,Tottenham Hotspur FC,Manchester City FC,FINISHED,AWAY_TEAM,21.0,0.0


In [50]:
# Setting initial point for Manchester United FC, Manchester City FC and Liverpool FC 

ManUtd['point'] = 0
ManCity['point'] = 0
Liv['point'] = 0

# # Code for generating point based on each match result which those threes attended to
ManUtd.loc[(ManUtd['hometeam']=='Manchester United FC') & (ManUtd['winner']=='HOME_TEAM'),'point'] = 3
ManUtd.loc[(ManUtd['hometeam']=='Manchester United FC') & (ManUtd['winner']=='AWAY_TEAM'),'point'] = 0
ManUtd.loc[(ManUtd['awayteam']=='Manchester United FC') & (ManUtd['winner']=='AWAY_TEAM'),'point'] = 3
ManUtd.loc[(ManUtd['awayteam']=='Manchester United FC') & (ManUtd['winner']=='HOME_TEAM'),'point'] = 0
ManUtd.loc[(ManUtd['winner']=='DRAW'),'point'] = 1

ManCity.loc[(ManCity['hometeam']=='Manchester City FC') & (ManCity['winner']=='HOME_TEAM'),'point'] = 3
ManCity.loc[(ManCity['hometeam']=='Manchester City FC') & (ManCity['winner']=='AWAY_TEAM'),'point'] = 0
ManCity.loc[(ManCity['awayteam']=='Manchester City FC') & (ManCity['winner']=='AWAY_TEAM'),'point'] = 3
ManCity.loc[(ManCity['awayteam']=='Manchester City FC') & (ManCity['winner']=='HOME_TEAM'),'point'] = 0
ManCity.loc[(ManCity['winner']=='DRAW'),'point'] = 1

Liv.loc[(Liv['hometeam']=='Liverpool FC') & (Liv['winner']=='HOME_TEAM'),'point'] = 3
Liv.loc[(Liv['hometeam']=='Liverpool FC') & (Liv['winner']=='AWAY_TEAM'),'point'] = 0
Liv.loc[(Liv['awayteam']=='Liverpool FC') & (Liv['winner']=='AWAY_TEAM'),'point'] = 3
Liv.loc[(Liv['awayteam']=='Liverpool FC') & (Liv['winner']=='HOME_TEAM'),'point'] = 0
Liv.loc[(Liv['winner']=='DRAW'),'point'] = 1


C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [51]:
# Manchester United FC, Manchester City FC, Liverpool FC point accumulation 

ManUtd['total_point'] = ManUtd['point'].cumsum()
ManCity['total_point'] = ManCity['point'].cumsum()
Liv['total_point'] = Liv['point'].cumsum()

C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Rizdi\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [52]:
# Cutting four tables and combine into one table

df_TotHam = TotHam[['matchday','total_point']].reset_index(drop=True).rename(columns={'total_point': 'Tottenham Hostpur FC'})
df_Liv = Liv[['matchday','total_point']].reset_index(drop=True).rename(columns={'total_point': 'Liverpool FC'})
df_ManUtd = ManUtd[['matchday','total_point']].reset_index(drop=True).rename(columns={'total_point': 'Manchester United FC'})
df_ManCity = ManCity[['matchday','total_point']].reset_index(drop=True).rename(columns={'total_point': 'Manchester City FC'})


In [53]:
result = pd.merge(df_TotHam,df_Liv, on = 'matchday')
result2 = pd.merge(result, df_ManUtd, on = 'matchday')
result3 = pd.merge(result2, df_ManCity, on = 'matchday')
result3

,matchday,Tottenham Hostpur FC,Liverpool FC,Manchester United FC,Manchester City FC
0,1,3.0,3,3,3
1,2,6.0,6,3,6
2,3,9.0,9,3,7
3,4,9.0,12,6,10
4,5,9.0,15,9,13
5,6,12.0,18,10,16
6,7,15.0,19,10,19
7,8,18.0,20,13,20
8,9,21.0,23,14,23
9,10,21.0,26,17,26


In [55]:
# Importing plotly for team position plotting

import plotly.plotly as py
from plotly.grid_objs import Grid, Column
import plotly.tools
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Plotting clubs with points

trace4 = go.Scatter(
x = result3['matchday'],
y = result3['Tottenham Hostpur FC'],
name = 'Tottenham Hostpur FC'
)

trace5 = go.Scatter(
x = result3['matchday'],
y = result3['Manchester United FC'],
name = 'Manchester United FC'
)
trace6 = go.Scatter(
x = result3['matchday'],
y = result3['Manchester City FC'],
name = 'Manchester City FC'
)
trace7 = go.Scatter(
x = result3['matchday'],
y = result3['Liverpool FC'],
name = 'Loserpool FC'
)


data = [trace4, trace5, trace6, trace7]

plotly.offline.iplot(data, filename='basic-line')